# Compute alternative variables
## New studies should use the versions implemented in CMSSW

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
import matplotlib
import imblearn
from sklearn.model_selection import train_test_split
import collections
from sklearn.metrics import roc_curve, auc
from collections import Counter
pd.options.mode.chained_assignment = None

In [2]:
workdir=os.getcwd()
samples = ['elec','PU', 'QCD'] #list of samples to process, should correspond to file naming 
savedir = workdir + '/New_vars' #output dir
os.makedirs(savedir, exist_ok=True)
data_dir=workdir+'/preprocessing/200PU_2806' #path to the preprocessed ntuples

In [21]:
# choice of variables for each sample type

columns={}
df_samples={}

columns['elec']=['genpart_exeta', 'genpart_pt', 'cl3d_pt', 'cl3d_eta', 'cl3d_showerlength', 'cl3d_coreshowerlength',
 'cl3d_firstlayer', 'cl3d_maxlayer', 'cl3d_seetot', 'cl3d_seemax', 'cl3d_spptot', 'cl3d_sppmax', 'cl3d_szz',
 'cl3d_srrtot', 'cl3d_srrmax', 'cl3d_srrmean', 'cl3d_emaxe', 'cl3d_hoe', 'cl3d_meanz', 'cl3d_layer10', 'cl3d_layer50',
 'cl3d_layer90', 'cl3d_ntc67', 'cl3d_ntc90', 'layer', 'sample',  'matches',      
        ]


columns['QCD'] =['event', 'genjet_n', 'genjet_energy', 'genjet_pt', 'genjet_eta',
       'genjet_phi', 'cl3d_pt', 'cl3d_energy', 'cl3d_eta',
       'cl3d_phi', 
       'cl3d_showerlength', 'cl3d_coreshowerlength', 'cl3d_firstlayer',
       'cl3d_maxlayer', 'cl3d_seetot', 'cl3d_seemax', 'cl3d_spptot',
       'cl3d_sppmax', 'cl3d_szz', 'cl3d_srrtot', 'cl3d_srrmax', 'cl3d_srrmean',
       'cl3d_emaxe', 'cl3d_hoe', 'cl3d_meanz', 'cl3d_layer10', 'cl3d_layer50',
       'cl3d_layer90', 'cl3d_ntc67', 'cl3d_ntc90', 'layer', 'deta', 'dphi',
       'deltar', 'matches',]


columns['PU'] = ['event', 'cl3d_pt', 'cl3d_energy', 'cl3d_eta', 'cl3d_phi', 
       'cl3d_showerlength', 'cl3d_coreshowerlength', 'cl3d_firstlayer',
       'cl3d_maxlayer', 'cl3d_seetot', 'cl3d_seemax', 'cl3d_spptot',
       'cl3d_sppmax', 'cl3d_szz', 'cl3d_srrtot', 'cl3d_srrmax', 'cl3d_srrmean',
       'cl3d_emaxe', 'cl3d_hoe', 'cl3d_meanz', 'cl3d_layer10', 'cl3d_layer50',
       'cl3d_layer90', 'cl3d_ntc67', 'cl3d_ntc90', 'layer',]


for s in samples:
    df_samples[s]= pd.read_csv(data_dir+'/{}.csv'.format(s), usecols=columns[s], low_memory=True)
print('done')

done


# Preprocessing

In [23]:
# select cuts on data

genptcut=20 # cut on gen pt in GeV
cl3dptcut=5 # cut on cluster pt in GeV
PU_cut = 20 #cut on PU cluster pt in GeV
etamin=1.6 #cut on cluster eta
etamax=2.9 #cut on cluster eta


In [53]:
%%time

count=Counter
df_cut={}

#select clusters based on eta and pt
for s in samples:
    df_samples[s]['abseta']=np.abs(df_samples[s]['cl3d_eta'])
    print(s)
    if s=='elec': #add samples that have genpart here
        sel = ((np.abs(df_samples[s]['genpart_exeta'])>etamin) & (np.abs(df_samples[s]['genpart_exeta'])<etamax )
               & (df_samples[s]['genpart_pt']>genptcut)&  (df_samples[s]['cl3d_pt']>cl3dptcut) 
               &(df_samples[s]['abseta']>etamin) & (df_samples[s]['abseta']<etamax ))
        df_cut[s]=df_samples[s][sel]
        df_cut[s].dropna(inplace=True)

    elif s!='elec':
        sel = ((np.abs(df_samples[s]['cl3d_eta'])>etamin) & (np.abs(df_samples[s]['cl3d_eta'])<etamax )&(df_samples[s]['cl3d_pt']>PU_cut))
        df_cut[s]=df_samples[s][sel]
        df_cut[s].dropna(inplace=True)


elec


/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


PU


/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


QCD
CPU times: user 964 ms, sys: 143 ms, total: 1.11 s
Wall time: 1.1 s


In [54]:
%%time
#extract pt per layer info for computing the new variables

def tolist(x):
    x.test=x.layer[1:-1].split(',')
    return x.test

def layering (x):
    return float(x.layer_pt[n])

for s in samples:
    print('Layering {}:'.format(s))
    #layer_pt preproc
    df_cut[s]['layer_pt']=df_cut[s].apply(tolist, axis=1)
    df_cut[s].drop('layer', axis=1, inplace=True)
    print("done")
    n_layers=len(df_cut['elec']['layer_pt'].iloc[0])
    #print(n_layers)
    layer_columns=[]

    for n in range(n_layers):
        print('layering: {}/{}\r'.format(n+1,n_layers),end='', flush=True)
        df_cut[s]['layer_{}'.format(n)]=df_cut[s].apply(layering, axis=1)
        layer_columns.append('layer_{}'.format(n))
    print("done layering")  
    


/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


done


/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


/opt/exp_soft/llr/python/3.7.0/el7/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


done layering36
done
done layering36
done
done layering36
CPU times: user 4min 36s, sys: 15.3 s, total: 4min 51s
Wall time: 4min 52s


# Build alternative variables (easier to compute with HGCAL primitives)

In [55]:
#bitmap functions
def ebm0(x):
    ebm=[]
    thr=0
    for i in range(1,nLayerEcal+1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
def ebm1(x):
    ebm=[]
    thr=1
    for i in range(1,nLayerEcal+1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
def hbm(x):
    ebm=[]
    thr=0
    for i in range(nLayerEcal+1,n_layers):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))

def reverse_ebm0(x):
    ebm=[]
    thr=0
    for i in range(nLayerEcal, 0, -1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
def reverse_ebm1(x):
    ebm=[]
    thr=1
    for i in range(nLayerEcal, 0, -1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))
def reverse_hbm(x):
    ebm=[]
    thr=0
    for i in range(n_layers-1,nLayerEcal-1, -1):
        #print(float(x.layer_pt[i])>thr)
        ebm.append(int(float(x.layer_pt[i])>thr)) 
    #print(ebm)
    return(np.array(ebm).dot(2**np.arange(len(ebm))[::-1]))

In [58]:
for s in samples:
    print('Computing variables for ', s)

    #std deviations replaced with variances
    df_cut[s]['varee']=df_cut[s]['cl3d_seetot']**2
    df_cut[s]['varpp']=df_cut[s]['cl3d_spptot']**2
    df_cut[s]['varzz']=df_cut[s]['cl3d_szz']**2
    df_cut[s]['varrr']=df_cut[s]['cl3d_srrtot']**2

    #Replace H/E by EoT: pt(Ecal)/pt(Tot), always defined
    nLayerEcal= 14
    sumE=0
    sumT=0
    for i in range(n_layers):
        sumT+=df_cut[s]['layer_{}'.format(i)]
    for i in range(1,nLayerEcal+1):
        sumE+=df_cut[s]['layer_{}'.format(i)]
    df_cut[s]['EoT']=sumE/sumT

    #fraction of pt in first x  layers and last x layers
    maxfirst = 5
    maxlast=10
    #firstX
    for n in range(1,maxfirst+1):
        Sum=0
        for i in range(1,n+1):
            Sum+=df_cut[s]['layer_{}'.format(i)]
        df_cut[s]['first_{}'.format(n)]= Sum/sumT
    #lastX
    for n in range(1,maxlast):
        Sum=0
        #print(n, ' layers')
        for i in range(n_layers - n, n_layers):
            #print(n,i)
            Sum+=df_cut[s]['layer_{}'.format(i)]
        #print('last_{}'.format(n), sum/sumT)
        df_cut[s]['last_{}'.format(n)]= Sum/sumT
    #first HCAL X
    for n in range(1,maxfirst+1):
        Sum=0
        for i in range(1,n+1):
            #print('layer_{}'.format(nLayerEcal+i))
            Sum+=df_cut[s]['layer_{}'.format(nLayerEcal+i)]
        df_cut[s]['firstHcal_{}'.format(n)]= Sum/sumT
    
    
    # Emaxx : pt in x layers around Elec max layer (5)
    maxpos=5 #typical electron cluster max shower on 6th layer

    #print('emax_1')
    Sum=0
    Sum+=df_cut[s]['layer_{}'.format(maxpos)]
    df_cut[s]['Emax_1']=Sum/sumT

    #print('emax_2L')
    Sum=0
    for i in range(maxpos-1, maxpos+1):
        #print(i)
        Sum+=df_cut[s]['layer_{}'.format(i)]
        df_cut[s]['Emax_2L']=Sum/sumT
    
    #print('emax_2R')
    Sum=0
    for i in range(maxpos, maxpos+2):
        #print(i)
        Sum+=df_cut[s]['layer_{}'.format(i)]
        df_cut[s]['Emax_2R']=Sum/sumT

    #print('emax_3')
    Sum=0
    for i in range(maxpos-1, maxpos+2):
        #print(i)
        Sum+=df_cut[s]['layer_{}'.format(i)]
        df_cut[s]['Emax_3']=Sum/sumT
    #print('emax_4G')
    Sum=0
    for i in range(maxpos-2, maxpos+2):
        #print(i)
        Sum+=df_cut[s]['layer_{}'.format(i)]
        df_cut[s]['Emax_4L']=Sum/sumT
    #print('emax_4R')
    Sum=0
    for i in range(maxpos-1, maxpos+3):
        #print(i)
        Sum+=df_cut[s]['layer_{}'.format(i)]
        df_cut[s]['Emax_4R']=Sum/sumT
    #print('emax_5')
    Sum=0
    for i in range(maxpos-2, maxpos+3):
        #print(i)
        Sum+=df_cut[s]['layer_{}'.format(i)]
        df_cut[s]['Emax_5']=Sum/sumT
        
    #Bitmaps of energy in CE-E/CE-H, threshold not implemented yet in CMSSW
   

    df_cut[s]['ebm0']=df_cut[s].apply(ebm0, axis=1)
    df_cut[s]['ebm1']=df_cut[s].apply(ebm1, axis=1)
    df_cut[s]['hbm']=df_cut[s].apply(hbm, axis=1) 
    
        
    df_cut[s]['reverse_ebm0']=df_cut[s].apply(reverse_ebm0, axis=1)
    df_cut[s]['reverse_ebm1']=df_cut[s].apply(reverse_ebm1, axis=1)
    df_cut[s]['reverse_hbm']=df_cut[s].apply(reverse_hbm, axis=1)
print('done')

Computing variables for  elec
Computing variables for  PU
Computing variables for  QCD
done


# Save files

In [62]:
for s in samples:
    df_cut[s].to_csv(savedir+'/{}_df.csv'.format(s))